# Matrix Multiply
 - Write matrix multiply using Python Lists
 - Optimize matrix multiplication using numpy

### Python List Matrix Multiply
Write a function to do matrix multiply of python lists, also include tests to ensure nested lists are of equal length, and matrices are of compatible size to be matrix multiplied

In [1]:
def check_valid_matrix(mat):
    '''Check matrix has same number of elements in all rows'''
    cols = len(mat[0])
    for c in mat:
        assert cols == len(c)
        
              
def get_shape(mat):
    '''Return a tuple of (number of rows, number of columns) of a matrix'''
    n_rows = len(mat)
    n_cols = len(mat[0])
    return (n_rows, n_cols)


def init_result(shape):
    '''Initialise Result Matrix to be all zeros'''
    res = []
    for i in range(shape[0]):
        row = []
        for j in range(shape[1]):
            row.append(0)
        res.append(row)
    return res


def get_col_vector(mat, col_i):
    '''Take a 2D list and a column index and return a 1D list of all values in that column'''
    col_vec = []
    for row in mat:
        col_vec.append(row[col_i])
    return col_vec


def matmul(a, b):
    '''matrix multiply matrix a * matrix b'''
    check_valid_matrix(a)
    check_valid_matrix(b)
    assert get_shape(a)[1] == get_shape(b)[0]
    result_shape = get_shape(a)[0], get_shape(b)[1]
    res = init_result(result_shape)
    for i, row in enumerate(a):
        for j in range(result_shape[1]):
            col = get_col_vector(b, j)
            r = 0
            for e1, e2 in zip(row, col):
                r = r + e1*e2
            res[i][j] = r
    return res

Test by multiplying a 4x3 matrix with a 3x2 matrix

In [2]:
a = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
b = [[2, 4], [6, 8], [10, 12]]

In [3]:
matmul(a, b)

[[44, 56], [98, 128], [152, 200], [206, 272]]

Result is 4x2 matrix as expected and looks good

### Optimize using numpy

In [4]:
import numpy as np

In [5]:
def np_matmul(a, b):
    '''matrix multiply in numpy using 3x explicit for loops'''
    res = np.zeros([a.shape[0], b.shape[1]])
    for i in range(a.shape[0]):
        for j in range(b.shape[1]):
            for k in range(a.shape[1]):
                res[i, j] += a[i, k] * b[k, j]
    return res
    

In [6]:
# convert lists a and b to numpy arrays so numpy array methods can be used
a, b = np.array(a), np.array(b)
# test result
np_matmul(a, b)

array([[ 44.,  56.],
       [ 98., 128.],
       [152., 200.],
       [206., 272.]])

Result looks good.

Now use numpy element wise multiply and sum to replace the inner most loop.

In [8]:
def np_matmul2(a, b):
    res = np.zeros([a.shape[0], b.shape[1]])
    for i in range(a.shape[0]):
        for j in range(b.shape[1]):
            res[i, j] = (a[i, :] * b[:, j]).sum()
    return res
    

Now use broadcasting to replace the next inner most loop so that python for loop only needs to be run over the rows of the first matrix being multiplied

In [9]:
def np_matmul3(a, b):
    res = np.zeros([a.shape[0], b.shape[1]])
    for i in range(a.shape[0]):
        res[i] = (a[i][:, None] * b).sum(0)
    return res

Make larger matrices so that times can be compared between different implementations

In [10]:
c = np.arange(15000).reshape([50, 300])
d = np.arange(6000).reshape([300, 20])

In [12]:
%timeit -n 10 r = matmul(c, d)
print(f'\n{r[0][:3]}\n...')

122 ms ± 3.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

[179101000, 179145850, 179190700]
...


In [14]:
%timeit -n 10 r = np_matmul(c, d)
print(f'\n{r[0, :3]}\n...')

186 ms ± 4.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

[1.7910100e+08 1.7914585e+08 1.7919070e+08]
...


In [15]:
%timeit -n 10 r = np_matmul2(c, d)
print(f'\n{r[0, :3]}\n...')

12.1 ms ± 2.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

[1.7910100e+08 1.7914585e+08 1.7919070e+08]
...


In [17]:
%timeit -n 10 r = np_matmul3(c, d)
print(f'\n{r[0, :3]}\n...')

3.06 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

[1.7910100e+08 1.7914585e+08 1.7919070e+08]
...


In [18]:
%timeit -n 10 r = np.matmul(c, d) # also can use: c@d
print(f'\n{r[0, :3]}\n...')

375 µs ± 51.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

[1.7910100e+08 1.7914585e+08 1.7919070e+08]
...


In [20]:
import pandas as pd

In [21]:
results = pd.DataFrame({'implementation': ['python', 'np_matmul', 'np_matmul2', 'np_matmul3', 'np_native']})

In [22]:
results['time'] = [122, 186, 12,3, 0.4]

In [26]:
results['improvement'] = results['time'].apply( lambda x: 122/x )

In [30]:
results['comments'] = ['Implementing with Python Lists', 
                       'Using 3 for loops over numpy arrays is slower',
                       'Removing inner loop is 10x faster than Python',
                       'Removing next inner loop 40x faster',
                       'Numpy\'s implementation 300x faster']

In [31]:
results

,implementation,time,improvement,comments
0,python,122.0,1.000000,Implementing with Python Lists
1,np_matmul,186.0,0.655914,Using 3 for loops over numpy arrays is slower
2,np_matmul2,12.0,10.166667,Removing inner loop is 10x faster than Python
3,np_matmul3,3.0,40.666667,Removing next inner loop 40x faster
4,np_native,0.4,305.000000,Numpy's implementation 300x faster
